In [54]:
import pandas as pd
import numpy as np
import string

import spacy #load spacy
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
stops = nlp.Defaults.stop_words


In [66]:
df  = pd.read_csv('C:/Users/ksa/OneDrive - Delta Partners/Documents/Feedback.csv', header=None)

In [67]:
df.head()

,0
0,fun work most of the time
1,Very Little Work Life Balance
2,"organizations is an amazing place to work, lea..."
3,"Bright and engaging people, amazing financial ..."
4,perks (lots of free stuff)


In [99]:
df.columns =['Feedback']
df.head()

ValueError: Length mismatch: Expected axis has 6 elements, new values have 1 elements

# Text Preprocessing

In [69]:
# Removing null values 
df.isnull().sum()
df.dropna(inplace = True)

In [70]:
# Remove white spaces and blanks
blanks = []

for fb in df.itertuples():
    #     (index, label and revies)
    if type(fb) == str:
        if rv.isspace():
            blanks.append(i)


In [71]:
len(blanks)

0

In [72]:
df.Feedback = df.Feedback.astype(str)

In [73]:
df.head()

,Feedback
0,fun work most of the time
1,Very Little Work Life Balance
2,"organizations is an amazing place to work, lea..."
3,"Bright and engaging people, amazing financial ..."
4,perks (lots of free stuff)


In [75]:
#convert tweets to lower case
df['preprocess_data'] = df['Feedback'].str.lower()
#url removes
df['preprocess_data'] = df['preprocess_data'].str.replace(r'(https|http)?:\/(\w|\.|\/|\?|\=|\&|\%)*\b','')
df['preprocess_data'] = df['preprocess_data'].str.replace(r'www\.\S+\.com','')


#removes retweets & cc
df['preprocess_data'] = df['preprocess_data'].str.replace(r'rt|cc', '')

#hashtags removes
df['preprocess_data'] = df['preprocess_data'].str.replace(r'#\S+', '')

#user mention removes
df['preprocess_data'] = df['preprocess_data'].str.replace(r'@\S+', '')

#emoji 
df['preprocess_data'] = df['preprocess_data'].str.replace(r'[^\x00-\x7F]+', '')

#html tags
df['preprocess_data'] = df['preprocess_data'].str.replace(r'<.*?>', '')

#removes extra spaces
df['preprocess_data'] = df['preprocess_data'].str.replace(r' +', ' ')

#punctuation
df['preprocess_data'] = df['preprocess_data'].str.replace('[{}]'.format(string.punctuation), '')



In [76]:
#  Funtion to convert into lowercase, remove stop words
def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

In [77]:
# Apply function on the data frame text col
df['Feedback_clean'] = df['preprocess_data'].apply(normalize, lowercase=True, remove_stopwords=True)

# Sentiment Analysis

In [78]:
# import sentiment analysis libraries
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ksa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [79]:
sid = SentimentIntensityAnalyzer()

In [80]:
# Getting polarity scores of full dataset
df['scores'] = df['Feedback_clean'].apply(lambda feedback: sid.polarity_scores(feedback))

In [84]:
df['compound'] = df['scores'].apply(lambda d:d['compound'])

In [86]:
df['comp_score'] = df['compound'].apply(lambda score: 'pos' if score >=0 else 'neg')

In [98]:
df.Feedback[5]

"If you aren't in shape it gets tough and you just become physically and mentally exhausted. It is a lot of walking."

# Extracting Important Topics from the feedbacks

In [100]:
from sklearn.feature_extraction.text import CountVectorizer

In [101]:
cv = CountVectorizer(max_df = .9, min_df = 2, stop_words='english')

In [109]:
dtm = cv.fit_transform(df['Feedback_clean'])

In [110]:
dtm

<1994x1345 sparse matrix of type '<class 'numpy.int64'>'
	with 15057 stored elements in Compressed Sparse Row format>

In [103]:
from sklearn.decomposition import LatentDirichletAllocation

In [115]:
# Pass parameters
# N_components = Number of Topics randomly

LDA = LatentDirichletAllocation(n_components=3, random_state=42)

In [117]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [118]:
# Grab vocabulary of words
len(cv.get_feature_names())

1345

In [119]:
# Grab the topics
len(LDA.components_)

3

In [142]:
# Grab the highest probability word for each topic
for i, topic in enumerate(LDA.components_):
    print(f"The top 15 words for topic #{i}")
    print([cv.get_feature_names()[index] for index in topic.argsort()[-15:]])
    print('\n')
    print('\n')

The top 15 words for topic #0
['food', 'job', 'perk', 'company', 'time', 'environment', 'place', 'lot', 'organization', 'pay', 'benefit', 'people', 'good', 'great', 'work']




The top 15 words for topic #1
['oppounity', 'career', 'project', 'grow', 'fast', 'challenge', 'sma', 'culture', 'pron', 'team', 'organization', 'work', 'lot', 'people', 'company']




The top 15 words for topic #2
['thing', 'job', 'high', 'lot', 'like', 'hard', 'employee', 'day', 'management', 'big', 'company', 'time', 'long', 'work', 'hour']






In [121]:
# Get the topic results against each document vector
topics_results = LDA.transform(dtm)

In [123]:
# assigning topic to the base table for each text
df['Topic'] = topics_results.argmax(axis = 1)

In [125]:
df.head()

,Feedback,preprocess_data,Feedback_clean,scores,compound,comp_score,Topic
0,fun work most of the time,fun work most of the time,fun work time,"{'neg': 0.0, 'neu': 0.377, 'pos': 0.623, 'comp...",0.5106,pos,0
1,Very Little Work Life Balance,very little work life balance,little work life balance,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,0
2,"organizations is an amazing place to work, lea...",organizations is an amazing place to work lear...,organization amaze place work learn grow quick...,"{'neg': 0.049, 'neu': 0.601, 'pos': 0.35, 'com...",0.8591,pos,0
3,"Bright and engaging people, amazing financial ...",bright and engaging people amazing financial a...,bright engage people amaze financial nonfinanc...,"{'neg': 0.0, 'neu': 0.387, 'pos': 0.613, 'comp...",0.9647,pos,0
4,perks (lots of free stuff),perks lots of free stuff,perk lot free stuff,"{'neg': 0.0, 'neu': 0.476, 'pos': 0.524, 'comp...",0.5106,pos,0


In [136]:
df.groupby(['comp_score', 'Topic']).agg(['count'])

Feedback preprocess_data Feedback_clean scores compound
                    count           count          count  count    count
comp_score Topic                                                        
neg        0           73              73             73     73       73
           1          100             100            100    100      100
           2          187             187            187    187      187
pos        0          859             859            859    859      859
           1          448             448            448    448      448
           2          327             327            327    327      327

In [141]:
df.loc[df.comp_score =='neg',['Feedback', 'Topic']].head(2)

,Feedback,Topic
5,If you aren't in shape it gets tough and you j...,2
8,As a virtual employee there is little opportun...,1
